Скачайте файл curated_gene_disease_associations.tsv и gene.fasta на ПК. В первом файле содержится список генов и ассоциированных с ними болезней. Во втором содержится последовательность гена, который приведен в анализе Task1.ipynb - вам нужно будет скачать последовательность выбранного вами гена с его страницы в NCBI Gene.

Что требуется сделать:

* Выберите любое из заболеваний, которое есть в списке.
В отдельную таблицу извлеките все гены, которые связаны с избранным заболеванием.
* Для одного из генов извлеките отдельно информацию о 2–3 характеристиках на выбор, а также добавьте вырезанную по координатам последовательность гена.
* В таблицу генов добавьте следующую информацию для каждого гена: аминокислотную последовательность, количество транскриптов, количество аминокислот для самой длинной изоформы белка.

In [71]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass


In [1]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "aspirantsv1@gmail.com"
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
dis = pd.read_table('gene_disease_associations.tsv')
dis

,gene_pli,score,diseaseid,gene_dsi,protein_class_name,geneid,disease_class,disease_type,uniprotid,disease_class_name,disease_name,el,year_initial,source,ei,gene_dpi,protein_class,year_final,gene_symbol,disease_semantic_type
0,1.643200e-05,1.00,C0001080,0.391,Kinase,2261,C16;C05,disease,P22607,"Congenital, Hereditary, and Neonatal Diseas...",Achondroplasia,definitive,1994.0,CURATED,0.974,0.846,DTO_03300101,2020.0,FGFR3,Congenital Abnormality
1,1.040000e-05,0.80,C0000744,0.505,Transporter,4547,C16;C18,disease,P55157,"Congenital, Hereditary, and Neonatal Diseas...",Abetalipoproteinemia,NaN,1985.0,CURATED,0.982,0.846,DTO_05007405,2019.0,MTTP,Disease or Syndrome
2,9.973100e-01,0.80,C0001193,0.380,Kinase,2263,C16;C05,disease,P21802,"Congenital, Hereditary, and Neonatal Diseas...",Apert syndrome,NaN,1994.0,CURATED,0.976,0.808,DTO_03300101,2019.0,FGFR2,Congenital Abnormality
3,5.323500e-01,0.70,C0001418,0.236,Transcription factor,7157,C04,group,P04637,Neoplasms,Adenocarcinoma,NaN,1986.0,CURATED,0.975,0.962,DTO_05007542,2020.0,TP53,Neoplastic Process
4,8.460200e-01,0.69,C0001126,0.522,Transporter,6521,C16;C18;C13;C12,phenotype,P02730,"Congenital, Hereditary, and Neonatal Diseas...",Renal tubular acidosis,NaN,1996.0,CURATED,1.000,0.731,DTO_05007405,2013.0,SLC4A1,Disease or Syndrome
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,1.047800e-06,0.30,C0001430,0.578,NaN,6948,C04,group,P20062,Neoplasms,Adenoma,NaN,2007.0,CURATED,1.000,0.615,NaN,2007.0,TCN2,Neoplastic Process
454,2.095700e-32,0.30,C0001430,0.419,Transporter,9429,C04,group,Q9UNQ0,Neoplasms,Adenoma,NaN,2011.0,CURATED,1.000,0.885,DTO_05007405,2011.0,ABCG2,Neoplastic Process
455,1.000000e+00,0.30,C0001546,0.552,Epigenetic regulator,10014,F03,group,Q9UQL6,Mental Disorders,Adjustment Disorders,NaN,2007.0,CURATED,1.000,0.769,DTO_05007378,2007.0,HDAC5,Mental or Behavioral Dysfunction
456,8.919400e-01,0.30,C0001618,0.636,Enzyme modulator,2771,C04;C19,group,P04899,Neoplasms; Endocrine System Diseases,Tumors of Adrenal Cortex,NaN,NaN,CURATED,NaN,0.577,DTO_05007584,NaN,GNAI2,Neoplastic Process


In [74]:
dis.shape

(458, 20)

1. Выберите любое из заболеваний, которое есть в списке.
В отдельную таблицу извлеките все гены, которые связаны с избранным заболеванием.

In [117]:
adenoma_db = dis.loc[dis['disease_name'] == 'Adenoma']
adenoma_db

,gene_pli,score,diseaseid,gene_dsi,protein_class_name,geneid,disease_class,disease_type,uniprotid,disease_class_name,disease_name,el,year_initial,source,ei,gene_dpi,protein_class,year_final,gene_symbol,disease_semantic_type
53,9.999900e-01,0.40,C0001430,0.303,NaN,1499,C04,group,P35222,Neoplasms,Adenoma,NaN,1996.0,CURATED,0.959,0.885,NaN,2019.0,CTNNB1,Neoplastic Process
54,1.000000e+00,0.40,C0001430,0.373,NaN,324,C04,group,P25054,Neoplasms,Adenoma,NaN,1990.0,CURATED,0.967,0.962,NaN,2018.0,APC,Neoplastic Process
55,2.716000e-01,0.40,C0001430,0.318,NaN,3479,C04,group,P05019,Neoplasms,Adenoma,NaN,1992.0,CURATED,0.875,0.885,NaN,2019.0,IGF1,Neoplastic Process
56,3.357700e-04,0.40,C0001430,0.560,Ion channel,3762,C04,group,P48544,Neoplasms,Adenoma,NaN,2012.0,CURATED,1.000,0.577,DTO_01300327,2019.0,KCNJ5,Neoplastic Process
57,7.880300e-04,0.40,C0001430,0.320,Enzyme modulator,3845,C04,group,P01116,Neoplasms,Adenoma,NaN,1991.0,CURATED,0.944,0.923,DTO_05007584,2020.0,KRAS,Neoplastic Process
58,2.901500e-07,0.40,C0001430,0.406,Signaling,5617,C04,group,P01236,Neoplasms,Adenoma,NaN,1981.0,CURATED,0.981,0.885,DTO_05007599,2019.0,PRL,Neoplastic Process
59,5.590300e-01,0.40,C0001430,0.291,Signaling,596,C04,group,P10415,Neoplasms,Adenoma,NaN,1994.0,CURATED,0.950,0.885,DTO_05007599,2011.0,BCL2,Neoplastic Process
60,5.323500e-01,0.40,C0001430,0.236,Transcription factor,7157,C04,group,P04637,Neoplasms,Adenoma,NaN,1986.0,CURATED,0.911,0.962,DTO_05007542,2020.0,TP53,Neoplastic Process
61,3.044700e-15,0.40,C0001430,0.486,G-protein coupled receptor,7253,C04,group,P16473,Neoplasms,Adenoma,NaN,1993.0,CURATED,0.926,0.808,DTO_02300001,2013.0,TSHR,Neoplastic Process
62,3.616300e-01,0.40,C0001430,0.517,Chaperone,9049,C04,group,O00170,Neoplasms,Adenoma,NaN,2007.0,CURATED,1.000,0.769,DTO_05007613,2019.0,AIP,Neoplastic Process


2. Для одного из генов извлеките отдельно информацию о 2–3 характеристиках на выбор, а также добавьте вырезанную по координатам последовательность гена.

In [12]:
# Выбираем ген CTNNB1 - нашли в genecards Entrez номер, хромосому
handle = Entrez.efetch(db="gene", id="1499", rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print("Coordinates on the chromosome:", start, "-", end, "Sequence ID:", gi_id, "Strandness:", strand)

Coordinates on the chromosome: 41199505 - 41240443 Sequence ID: 568815595 Strandness: plus


In [147]:
gene_locus

<Element 'Entrezgene_locus' at 0x000001C43BB1E930>

In [78]:
#Выгружаем с NCBI нуклеотидную последовательность 3 хромосомы, для этого узнать ID хромосомы - NC_000003.12
handle = Entrez.efetch(db="nucleotide",
                       id="NC_000003.12", 
                       rettype = 'fasta',
                       retmode = 'text')
#Загружаем
chromosome = SeqIO.read(handle, 'fasta')

In [79]:
#А та ли это хромосома?
chromosome.description

'NC_000003.12 Homo sapiens chromosome 3, GRCh38.p14 Primary Assembly'

In [80]:
#Вырезаем из хромосомы ген, для этого узнаем локацию нужного гена. Для CTNNB1 это 41199505..41240443
geneCTNNB1 = chromosome.seq[41199505:41240443]
print(geneCTNNB1)
len(geneCTNNB1)

...


40938

3. В таблицу генов добавьте следующую информацию для каждого гена: аминокислотную последовательность, количество транскриптов, количество аминокислот для самой длинной изоформы белка.

In [152]:
read = SeqIO.read('gene_CTNNB1.fna', 'fasta')
read.seq

Seq('AAGCCTCTCGGTCTGTGGCAGCAGCGTTGGCCCGGCCCCGGGAGCGGAGAGCGA...TTC')

In [151]:
#Slicing a sequence
seq_sl = read.seq[4:100]
seq_sl

Seq('CTCTCGGTCTGTGGCAGCAGCGTTGGCCCGGCCCCGGGAGCGGAGAGCGAGGGG...CCC')

In [109]:
#Nucleotide sequences and (reverse) complements
read.seq[4:100].complement()

Seq('GAGAGCCAGACACCGTCGTCGCAACCGGGCCGGGGCCCTCGCCTCTCGCTCCCC...GGG')

In [110]:
m_rna = read.seq[4:100].transcribe()
m_rna

Seq('CUCUCGGUCUGUGGCAGCAGCGUUGGCCCGGCCCCGGGAGCGGAGAGCGAGGGG...CCC')

In [112]:
seq_sl == m_rna

False

Это ожидаемо, поскольку в РНК - урацил

In [111]:
prot = m_rna.translate()
prot

Seq('LSVCGSSVGPAPGAESEGRRRRRKV*GAASVP')

In [165]:
# Транслируем полную первичную структуру молекулы белка
m_rna = read.seq.transcribe()
prot = m_rna.translate()
prot

c:\Users\60053972\Работа\10.05.2022\Проекты jupyter\DS в медицине - рабочий стол\evn_study\lib\site-packages\Bio\Seq.py:3482: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('KPLGLWQQRWPGPGSGERGEAETEEGLRSSFSPRRAATAGRGRSDSRGGRSLFP...*FI')

In [167]:
len(m_rna)

40939

Длинна м-РНК составляет 40939 нуклеотидов. Мы постоили белок на ней. Но это неправильно. М-МРН в процессе сплайсинга уменьшается - остаются только интроны. Она становится гораздо меньше. По данным NCBI м-РНК содержит от 3074 до 22916 нуклеотидов. А готовый белок содержит от 720 до 781 аминокислоты. Посморим на м-РНК одного из самых длинных белков и на сам белок.

In [212]:
SeqIO.read('rna.fna', 'fasta')

SeqRecord(seq=Seq('AAGCCTCTCGGTCTGTGGCAGCAGCGTTGGCCCGGCCCCGGGAGCGGAGAGCGA...TTC'), id='NM_001904.4', name='NM_001904.4', description='NM_001904.4 CTNNB1 [organism=Homo sapiens] [GeneID=1499] [transcript=1]', dbxrefs=[])

In [193]:
SeqIO.read('rna.fna', 'fasta').translate()

SeqRecord(seq=Seq('KPLGLWQQRWPGPGSGERGEAETEEGLRSSFSPRRAATAGRGRSDSRGGRSLFP...*FI'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [213]:
len(SeqIO.read('rna.fna', 'fasta').translate())

c:\Users\60053972\Работа\10.05.2022\Проекты jupyter\DS в медицине - рабочий стол\evn_study\lib\site-packages\Bio\Seq.py:3482: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


1220

In [214]:
SeqIO.read('protein.faa', 'fasta')

SeqRecord(seq=Seq('MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSLSGKGNPEE...TDL'), id='NP_001895.1', name='NP_001895.1', description='NP_001895.1 CTNNB1 [organism=Homo sapiens] [GeneID=1499] [isoform=1]', dbxrefs=[])

In [215]:
len(SeqIO.read('protein.faa', 'fasta'))

781

In [3]:
data_table = pd.read_table('data_table.tsv')
data_table

,gene_id,gene_symbol,description,scientific_name,common_name,tax_id,genomic_range,orientation,location,gene_type,transcript_accession,transcript_name,transcript_length,transcript_cds_coords,protein_accession,isoform_name,protein_length,protein_name
0,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,NM_001330729.2,transcript variant 4,3488,NM_001330729.2:368-2692,NP_001317658.1,isoform 2,774,catenin beta-1
1,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,NM_001904.4,transcript variant 1,3661,NM_001904.4:215-2560,NP_001895.1,isoform 1,781,catenin beta-1
2,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,NM_001098209.2,transcript variant 2,3356,NM_001098209.2:215-2560,NP_001091679.1,isoform 1,781,catenin beta-1
3,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,NM_001098210.2,transcript variant 3,3197,NM_001098210.2:215-2560,NP_001091680.1,isoform 1,781,catenin beta-1
4,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,XM_006712985.2,transcript variant X9,3074,XM_006712985.2:215-2377,XP_006713048.1,isoform X3,720,catenin beta-1
5,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,XM_024453356.2,transcript variant X6,3593,XM_024453356.2:147-2492,XP_024309124.1,isoform X1,781,catenin beta-1
6,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,XM_047447481.1,transcript variant X7,3746,XM_047447481.1:300-2645,XP_047303437.1,isoform X1,781,catenin beta-1
7,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,XM_047447477.1,transcript variant X1,3734,XM_047447477.1:288-2633,XP_047303433.1,isoform X1,781,catenin beta-1
8,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,XM_017005738.2,transcript variant X3,3270,XM_017005738.2:288-2633,XP_016861227.1,isoform X1,781,catenin beta-1
9,1499,CTNNB1,catenin beta 1,Homo sapiens,human,9606,NC_000003.12:41199505-41240443;NC_060927.1:412...,+;+,chr 3,PROTEIN_CODING,XM_047447483.1,transcript variant X10,3147,XM_047447483.1:288-2450,XP_047303439.1,isoform X3,720,catenin beta-1


In [6]:
len(data_table.protein_length)

14

Мы видим, что белок, полученный при трансляции м-РНК длиннее настоящего белка из базы NCBI. По видимому на этапах синтеза на рибосомах белок также не на всей м-РНК

In [7]:
tab = {
    'id': 1499,
    'accession': 'NC_000003',
    'version': 12,
    'interval_from': 41199505,
    'interval_to': 41240443,
    'prot.seq.example': [''.join(SeqIO.read('protein.faa', 'fasta'))],
    'gr_acid_num': data_table.protein_length.max(),
    'num_of_transcripts': len(data_table.protein_length)
}

In [8]:
pd.DataFrame(tab).T.rename(columns={0: 'CTNNB1'})

,CTNNB1
id,1499
accession,NC_000003
version,12
interval_from,41199505
interval_to,41240443
prot.seq.example,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...
gr_acid_num,781
num_of_transcripts,14
